Q1:How to Github and Why do we use Jupyter and Pycharm?<br />
A1：Jupyter易于交互，Pycharm易于调试

Q2:What's the Probability Model?<br />
A2:概率模型输出事件发生的概率。从数学的角度来解释，概率模型给出了一个函数$f(x_1,x_2,x_3...x_n)$，当给定随机变量$x_1,x_2,x_3...x_n$时，模型将输出一个概率$P$，且有$0\leqslant P\leqslant 1$.

Q3:Can you came up with some sceneraies at which we could use Probability Model?<br />
A3:分类问题都用到了概率模型.

Q4:Why do we use probability and what's the difficult points for programming based on parsing and pattern match?<br />
A4:用概率表示能有一个直观的可能性印象，而且便于结果之间的比较；模式匹配的难点在于规则有限，问句变化不够灵活.

Q5:What's the Language Model?<br />
A5:语言模型，输入一句话，输出这句话合理的概率.

Q6:Can you came up with some sceneraies at which we could use Language Model?<br />
A6:对文本进行语义分析；语音识别；机器翻译.

Q7:What's the 1-gram language model?<br />
A7:以一个词为单位，计算这个词出现的可能性. 假设用1-gram模型计算句子S出现的合理性，经过分词后$S=(w_1,w_2,w_3,...,w_n)$，则$$P(S)=\prod_{i=1}^nP(w_i)$$.

Q8:What's the disadvantages and advantages of 1-gram language model?<br />
A8:缺点：模型效果依赖于分词效果;忽略了当前词与其相邻词之间的关联关系，词出现的概率仅和当前词在语料库中出现的概率有关.<br />
    优点：模型简单，易于计算.

Q9:What't the 2-gram model?<br />
A9:计算当前词出现的概率时，假设仅与前一个词相关.假设用2-gram模型计算句子S出现的合理性，经过分词后$S=(w_1,w_2,w_3,...,w_n)$，则$$P(S)=\prod_{i=2}^nP(w_i|w_{i-1})$$.

Q10:what's the web crawler, and can you implement a simple crawler?<br />
A10:web crawler，从网页的源码中抓取所需要的信息

Q11:There may be some issues to make our crwaler programming difficult, what are these, and how do we solve them?<br />
A11:访问网页时，需要输入验证码，一般来说可以对验证码进行图像识别<br />
    需解析网页的地址发生变化<br />
    IP限制，可以设置一个IP代理池，每次访问时随机设置代理
    
Q12:What't the Regular Expression and how to use?<br />
A12:正则表达式是通过自己构建模式，去寻找匹配这种模式的字符串.在python中利用re包使用正则表达式.


# N-Gram

实现n-gram的语言模型<br />
使用语料：zhwiki-20190401-pages-articles.xml.bz，以10M大小对语料进行分割，本程序使用其中10个语料文件<br />

In [16]:
from hanziconv import HanziConv
import re
import jieba
from collections import Counter
from functools import reduce

In [17]:
def token(string):
    regex = re.compile('<doc(.*)>')
    regex2 = re.compile('（）')
    regex3 = re.compile('《》')
    regex4 = re.compile('「')
    regex5 = re.compile('」')
    regex6 = re.compile('</doc>')
    string = regex.sub('',string)
    string = regex2.sub('',string)
    string = regex3.sub('',string)
    string = regex4.sub('',string)
    string = regex5.sub('',string)
    string = regex6.sub('',string)
    return ' '.join(re.findall('[\w\d]+',string))

In [18]:
def Covert_to_Simplified(string):
    return HanziConv.toSimplified(string)

In [19]:
def cut(string): 
    return list(jieba.cut(string))

In [20]:
def get_prob(word,frequence_sum,words_count):
    esp = 1/frequence_sum
    if word in words_count:
        return words_count[word]/frequence_sum
    else:
        return esp

In [21]:
def get_n_gram_prob(words,_n_gram_sum,_n_gram_counter):
    words = ''.join(words)
    esp = 1/_n_gram_sum
    if words in _n_gram_counter:
        return _n_gram_counter[words]/_n_gram_sum
    else:
        return esp

In [23]:
def language_model_n_gram(sentence,n,frequence_sum,words_count,_n_gram_sum,_n_gram_counter):
    probability = 1
    words = cut(sentence)
    for i,word in enumerate(words):
        if i<n:
            prob = get_prob(word,frequence_sum,words_count)
        else:
            previous = words[i-n:i]
            prob = get_n_gram_prob(previous+list(word),_n_gram_sum,_n_gram_counter)
        probability *= prob
    return probability

In [24]:
#处理wiki中文语料
def get_articles(size):
    all_articles = []
    for i in range(size):
        if i<10:
            file = 'extracted/AA/wiki_' + '0' + str(i)
        else:
            file = 'extracted/AA/wiki_' + str(i)
        with open(file,'r',encoding='utf8') as fl:
            all_articles += fl.readlines()
    all_articles = [token(str(s)) for s in all_articles if s.strip()!='']
    all_articles = [Covert_to_Simplified(s) for s in all_articles]
    return all_articles

In [38]:
def get_data(size):
    #使用语料文件数量
    all_articles = get_articles(size)
    text = ''
    for sentence in all_articles:
        text += sentence
    TEXT = text
    
    #对所有语料进行分词
    all_tokens = cut(TEXT)
    #得到有效的分词结果
    valid_tokens = [t for t in all_tokens if t.strip() and t!='n']
    
    #计数
    words_count = Counter(valid_tokens)
    frequence = [f for w,f in words_count.most_common()]
    frequence_sum = sum(frequence)
    
    return frequence_sum,words_count,valid_tokens

In [36]:
def get_n_gram_data(n,valid_tokens):
    #决定n-gram数目，并将分词结果以n进行组合
    all_n_gram_words = [''.join(valid_tokens[i:i+n]) for i in range(len(valid_tokens[:-n]))]
    _n_gram_sum = len(all_n_gram_words)
    _n_gram_counter = Counter(all_n_gram_words)
    
    return _n_gram_sum, _n_gram_counter

In [37]:
if __name__ == '__main__':
    
    size = 10
    n = 2
    
    frequence_sum,words_count,valid_tokens = get_data(size)
    _n_gram_sum, _n_gram_counter = get_n_gram_data(n,valid_tokens)
    
    need_compared = [
    "今天晚上没有太阳 今天晚上没有月亮",
    "你有什么一件 你有什么意见",
    "我去吃好吃的，今天 今天我去吃好吃的",
    "数学是一门学科 数学是一门学问",
    "点餐的是一只猫 点餐的是小明"
    ]

    for s in need_compared:
        s1,s2 = s.split()
        p1,p2 = language_model_n_gram(s1,n,frequence_sum,words_count,_n_gram_sum,_n_gram_counter), language_model_n_gram(s2,n,frequence_sum,words_count,_n_gram_sum,_n_gram_counter)

        better = s1 if p1>p2 else s2

        print('{} is more possible'.format(better))
        print('-'*4+'{} with probility {}'.format(s1,p1))
        print('-'*4+'{} with probility {}'.format(s2,p2))
    

今天晚上没有月亮 is more possible
----今天晚上没有太阳 with probility 1.700331517555341e-23
----今天晚上没有月亮 with probility 1.700331517555341e-23
你有什么意见 is more possible
----你有什么一件 with probility 2.785473316121813e-21
----你有什么意见 with probility 2.785473316121813e-21
今天我去吃好吃的 is more possible
----我去吃好吃的，今天 with probility 1.0096143975983879e-43
----今天我去吃好吃的 with probility 6.095342142292449e-37
数学是一门学问 is more possible
----数学是一门学科 with probility 7.237734644855822e-21
----数学是一门学问 with probility 7.237734644855822e-21
点餐的是小明 is more possible
----点餐的是一只猫 with probility 4.126069132229608e-30
----点餐的是小明 with probility 7.038639464504087e-23


Q:Compared to the previous learned parsing and pattern match problems. What's the advantage and disavantage of Probability Based Methods?<br />
A:优点：在语料库足够大的情况下，几乎能对所有句子进行合理性判断<br />
缺点：模型效果依赖于分词，在分词结果不够准确时，也会对结果造成一定的偏差

# How to solve OOV problem?

Q1: How did you solve this problem in your programming task?<br />
A1：给定一个默认值，当某个词不在语料库中时，默认词频为1

Q2: Read about the 'Turing-Good Estimator', can explain the main points about this method, and may implement this method in your programming task<br />
A2：Turing-Good estimator可以处理数据稀疏的问题